# 텐서

- 다차원 배열
- GPU를 이용해 연산 가능

In [1]:
import tensorflow as tf

In [4]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


텐서의 크기와 데이터 타입

In [5]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


## 텐서의 특징

### Numpy와의 차이점
- 텐서는 GPU, TPU와 같은 가속기 메모리 사용 가능
- 텐서는 불변성을 가짐

### 호환성
- 텐서플로우 연산에서 넘파이 배열은 자동으로 텐서로 변환됨
- 넘파이 연산에서 텐서는 자동으로 넘파이 연산으로 변환됨

텐서는 GPU 메모리에 저장되며, 넘파이는 호스트 메모리에 저장되기 때문에 이러한 변환이 항상 가능한 것은 아님. 변환시에는 GPU에서 호스트 메모리로 복사가 필요함

In [6]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


# GPU 

## 가속화
텐서플로우 연산은 GPU를 이용해 가속화됨.   
코드상에서 명시되어 있지 않는 경우에는 자동으로 CPU와 GPU 중 한가지를 이용해 연산을 진행함.

## 장치 이름
Tensor.device는 텐서를 구성하고 있는 호스트 장치의 이름을 제공함.  
텐서가 호스트의 N 번째 GPU에 놓여있는 경우 문자열은 GPU:<N>으로 나타남.

In [7]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0:  
True


## 장치 할당

개별 연산을 위한 장치에 배치(replacement).  
코드상에서 명시되지 않은 경우 자동으로 결정됨.  
코드를 통해 특정 장치에 명시적으로 배치하는 것이 가능함.

In [8]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 52.25ms
On GPU:
10 loops: 653.99ms
